# <p align='center'>Лабораторная работа №9</p>
## <p align='center'>Обнаружение лиц</p>

<p align='right'>Выполнил: студент гр. 5.306М Лаптев А.В.</p>

# Каскадный классификатор Хаара
Для использования каскадного классфикатора используется класс CascdeClassifier, в объект которого с помощью метода load() загружается предобученный каскадный классификатор. Единственным аргументом данного метода является filename - имя файла, который нужно загрузить (в строковом формате).

Непосредственно обнаружение выполняется с использованием метода detectMultiScale(), который возвращает граничные прямоугольники для обнаруженных лиц или глаз. Метод detectMulriScale() может принимать следующие аргументы:
* image: объект изображения;
* scaleFactor: определяет насколько уменьшается размер изображения при каждом изменении масштаба;
* minNeighbords: указывает сколько соседей должно быть у каждого прямоугольника-кандидата, чтобы он сохранился;
* flags: для нового объекта класса не используется;
* minSize: минимальный возможный размер объекта;
* maxSize: максимальный допустимый размер объекта.

В данном примере используется только первый аргумент, все остальные имеют значение по умолчанию.

Также, в коде, в качестве предобработки изображения используется метод equalizeHist(), который позволяет выровнять и увеличить контрастность изображений. У данного метода есть 2 аргумента:
* src: исходное изображение в оттенках серого;
* dst: выровненное изображение.

In [1]:
from __future__ import print_function
import cv2 as cv


def detectAndDisplay(frame):
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    frame_gray = cv.equalizeHist(frame_gray)
    #-- Detect faces
    faces = face_cascade.detectMultiScale(frame_gray)
    for (x,y,w,h) in faces:
        center = (x + w//2, y + h//2)
        frame = cv.ellipse(frame, center, (w//2, h//2), 0, 0, 360, (255, 0, 255), 4)
        faceROI = frame_gray[y:y+h,x:x+w]
        #-- In each face, detect eyes
        eyes = eyes_cascade.detectMultiScale(faceROI)
        for (x2,y2,w2,h2) in eyes:
            eye_center = (x + x2 + w2//2, y + y2 + h2//2)
            radius = int(round((w2 + h2)*0.25))
            frame = cv.circle(frame, eye_center, radius, (255, 0, 0 ), 4)
    cv.imshow('Capture - Face detection', frame)


face_cascade_name = '../resources/haarcascade_frontalface_alt.xml'
eyes_cascade_name = '../resources/haarcascade_eye_tree_eyeglasses.xml'

face_cascade = cv.CascadeClassifier()
eyes_cascade = cv.CascadeClassifier()

#-- 1. Load the cascades
if not face_cascade.load(cv.samples.findFile(face_cascade_name)):
    print('--(!)Error loading face cascade')
    exit(0)

if not eyes_cascade.load(cv.samples.findFile(eyes_cascade_name)):
    print('--(!)Error loading eyes cascade')
    exit(0)
camera_device = 0

#-- 2. Read the video stream
cap = cv.VideoCapture(camera_device)

if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)

while True:
    ret, frame = cap.read()

    if frame is None:
        print('--(!) No captured frame -- Break!')
        break

    detectAndDisplay(frame)

    if cv.waitKey(10) == 27:
        break

cv.destroyAllWindows()

В доработанном ниже примере не представлено никаких новых мкетодов для работы с изображениями. Здесь вокруг распознанного лица рисуется прямоугольник.

In [2]:
# Load Cascade filter 
face_cascade = cv.CascadeClassifier('../resources/haarcascade_frontalface_default.xml')

# Create the face detecting function 
def detect_face(img):
    img_2 = img.copy()
    face_rects = face_cascade.detectMultiScale(
        img.copy(), 
        scaleFactor=1.1,
        minNeighbors=4
    )
    
    for (x, y, w, h) in face_rects:
        cv.rectangle(
            img_2,
            (x, y),
            (x + w, y + h),
            (255, 255, 255),
            3
        )
        
    return img_2# Detect the face

# Load the image file and convert the color mode
avengers = cv.imread('../resources/avengers.jpg')
avengers = cv.cvtColor(avengers, cv.COLOR_BGR2GRAY)# Detect the face and plot the result
detected_avengers = detect_face(avengers)

cv.imshow('image',detected_avengers)
cv.waitKey(0)
cv.destroyAllWindows()